Import the required libraries

In [2]:
import cv2
import os
from bs4 import BeautifulSoup
import requests as request
import numpy as np

create Laura's images folder

In [3]:
folder='laura_images/'
nowatermarkfolder = 'laura_images/nowatermark/'
url = "https://ofinodafoto.fotop.com.br/fotos/eventos/busca/cat/75245/evento/85654"

In [4]:
#check if folder named images exists,  if not create one
if not os.path.exists(folder):
    os.makedirs(folder)
if not os.path.exists(nowatermarkfolder):
    os.makedirs(nowatermarkfolder)

Websscrap the url

In [5]:
page = request.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
  
# get all the images
images = soup.findAll('img')

#remove images where source has '/publico/'
images = [image for image in images if '/publico/' not in image['src']]

# get the src of the images
for image in images:
    #download the images to the folder images
    image_url = image['src']
    print(image_url)
    filename = image_url.split('/')[-1]
    print(filename)
    r = request.get(image_url, stream=True)
    
    if r.status_code == 200:
        with open(folder + filename, 'wb') as f:
            for chunk in r:
                f.write(chunk)
        print('saved')
    else:
        print('not saved')


In [8]:
def remove_watermark(image, fullname_save_path):
    image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask_inv = cv2.bitwise_not(mask)
    result = cv2.bitwise_and(image, image, mask=mask_inv)
    cv2.imwrite(fullname_save_path, result)

#remove watermark from first file found in folder laura_images and save to laura_images_no_watermark
for image in os.listdir(folder):
    fullname = folder + image
    fullname_save_path = nowatermarkfolder + image
    print(fullname, fullname_save_path)
    remove_watermark(fullname, fullname_save_path)
    break
    

laura_images/img_7175_1030347_85654_g.jpg laura_images/nowatermark/img_7175_1030347_85654_g.jpg


In [16]:
# load images from folder and convert them to np array
images = []
image_array = []
for image in os.listdir(folder):
    fullname = folder + image
    image_array.append(np.array(cv2.imread(fullname)))


watermark_rgb = (255, 255, 255)
for i in range(len(image_array)):
    for j in range(len(image_array[0])):
        if image_array[i][j] == watermark_rgb:
            image_array[i][j] = (255, 255, 255)

import skimage.io as io
for image in images:
    io.imsave(nowatermarkfolder+"watermarked_no_watermark.png", image)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()